In [2]:
import pandas as pd
import numpy as np
import csv
import scipy.io as sio
import pprint
import sklearn
from sklearn.cluster import KMeans

In [17]:
#Create a dict with each row representing student, and each column representing concept
#step 1: check total number of concepts
#step 2: create empty dict for all students and all concepts
#step 3: populate it with sid
#step 3, for each sid, check whether the concept was seen or not, 
# add one more dimension which is 1 or 0 whther the student got it right in the first attempt or not

file_path = "./cleaned_data.csv"

concepts = set()
sids = set()
with open(file_path, 'r') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for row in csvreader:
        c = row[19]
        s = row[1]
        concepts.add(c)
        sids.add(s)

sid_to_vector = {}
with open(file_path, 'r') as f:
    csvreader = csv.reader(f)
    next(csvreader)
    for row in csvreader:
        c = row[19]
        s = row[1]
        if s in sid_to_vector:
            v = sid_to_vector[s]
            v[c] = 1
        else:
            # First time seeing the student
            sid_to_vector[s] = {}
            # Not sparse; switch to sparse later
            for con in concepts:
                sid_to_vector[s][con] = 0
            sid_to_vector[s][c] = 1
                
# pprint.pprint(sid_to_vector)

# Convert sid_to_vector to (1) a dict linking sid to position
# and (2) a list of lists, where the list at position corresponds to sid

i = 0
concept_order = {}
for c in concepts:
    if c not in concept_order:
        concept_order[c] = i
    i += 1

num_concepts = len(concepts)
sid_to_index = {}
index_to_sid = {}
vectors = []
index = 0
for sid, v in sid_to_vector.iteritems():
    if sid not in sid_to_index:
        sid_to_index[sid] = index
        index_to_sid[index] = sid
        temp = np.zeros(num_concepts)        
        for concept, flag in v.iteritems():
            temp[concept_order[concept]] = flag
        vectors.append(temp)
    index += 1

# See sklearn documentation on KMeans
kmeans = KMeans(n_clusters=2, random_state=0).fit(vectors)

#Assign students to clusters
# for i in range(len(kmeans.labels_)):
#      print "Student %s is in cluster %d." % (index_to_sid[i], kmeans.labels_[i])

#print kmeans.cluster_centers_

cluster_dict = {}
for i in range(len(kmeans.labels_)):
    cluster_dict[index_to_sid[i]] = kmeans.labels_[i]+1

sio.savemat('../matlab/clustered_dict.mat', cluster_dict)

# with open('cluster_assignments.csv', 'w') as outfile:
#     csvwriter = csv.writer(outfile)
#     for i in range(len(kmeans.labels_)):
#         csvwriter.writerow([index_to_sid[i], kmeans.labels_[i]+1])
        
# Next steps:
# 1. Assign students to clusters
# 4. Do this with a lot more data

In [9]:
# See sklearn documentation on KMeans
kmeans = KMeans(n_clusters=3, random_state=0).fit(vectors)

print len(kmeans.labels_)
vectors

43


[array([ 1.,  1.,  1.,  0.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         0.,  1.,  0.,  1.,  0.,  1.,  0.,  0.,  1.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         1.,  0.,  0.,  0.,  0.,  0.]),
 array([ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         0.,  1.,  0.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,  0.,  1.,  1.,
         0.,  1.,  0.,  0.,  0.,  0.,  0.,  0.,  1.,  0.,  0.,  0.,  1.,
         1.,  1.,  1.,  0.,  0.,  0.]),
 array([ 1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,
         1.,  1.,  1.,  1.,  0.,  1.,  1.,  1.,  1.,  1.,  0.,  1.,  1.,
         0.,  1.,  1.,  1.,  0.,  0.,  1.,  0.,  1.,  0.,  0.,  0.,  1.,
         1.,  1.,  1.,  1.,  0.,  1.]),
 array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  0.,
         1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,
    